In [ ]:
# Keep Data By Take your picture
import cv2
import os

folder_name = "DataSet_Me/person1"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# โหลดตัวตรวจจับใบหน้า (Haarcascade)
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# เปิดกล้องวิดีโอ
cap = cv2.VideoCapture(0)

# ตัวนับจำนวนภาพที่แคป
img_count = 0

while True:
    # อ่านเฟรมจากกล้อง
    ret, frame = cap.read()

    if not ret:
        print("ไม่สามารถอ่านภาพจากกล้องได้")
        break

    # แปลงภาพเป็นระดับสีเทา (grayscale)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # ตรวจจับใบหน้าในเฟรม
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # วาดกรอบสี่เหลี่ยมรอบใบหน้าที่ตรวจพบ
    for x, y, w, h in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # แคปใบหน้าและบันทึกในโฟลเดอร์ DataSet_Me
        face_img = frame[y : y + h, x : x + w]
        img_count += 1
        img_path = os.path.join(folder_name, f"face_{img_count}.jpg")
        cv2.imwrite(img_path, face_img)
        print(f"บันทึกภาพ {img_path}")

    # แสดงผลเฟรมปัจจุบัน
    cv2.imshow("Keeping Data......", frame)

    # กด 'q' เพื่อออกจากลูป
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# ปิดกล้องและปิดหน้าต่าง
cap.release()
cv2.destroyAllWindows()

In [ ]:
#Check file
import os

data_dir = "DataSet_Me"
if not os.path.exists(data_dir):
    print(f"โฟลเดอร์ {data_dir} ไม่พบ")
else:
    subfolders = os.listdir(data_dir)
    print(f"โฟลเดอร์ย่อยใน {data_dir}: {subfolders}")

    for subfolder in subfolders:
        subfolder_path = os.path.join(data_dir, subfolder)
        if os.path.isdir(subfolder_path):
            files = os.listdir(subfolder_path)
            print(f"ไฟล์ใน {subfolder_path}: {files}")

In [ ]:
#Train AI By Tensorflow
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# กำหนดขนาดของภาพ
img_height = 128
img_width = 128

# กำหนดขนาด batch
batch_size = 32

# พาธไปยังโฟลเดอร์ข้อมูล
train_dir = "DataSet_Me"  # โฟลเดอร์ที่เก็บข้อมูลใบหน้าแบ่งเป็น person1 และ unknown

# สร้าง ImageDataGenerator พร้อมการเพิ่มข้อมูล (Data Augmentation)
datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # ปรับขนาดพิกเซลเป็น 0-1
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

# สร้าง data generator สำหรับข้อมูลฝึก
data_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),  # กำหนดขนาดภาพที่ต้องการ
    batch_size=batch_size,
    class_mode="binary",  # สำหรับการจำแนกประเภทสองคลาส (person1 และ unknown)
)

# สร้างโมเดล CNN
model = models.Sequential(
    [
        layers.Conv2D(
            32, (3, 3), activation="relu", input_shape=(img_height, img_width, 3)
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.Dense(1, activation="sigmoid"),  # ใช้ sigmoid สำหรับการจำแนกประเภทสองคลาส
    ]
)

# คอมไพล์โมเดล
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",  # ใช้ binary_crossentropy สำหรับสองคลาส
    metrics=["accuracy"],
)

# ฝึกโมเดล
history = model.fit(data_gen, epochs=20, steps_per_epoch=data_gen.samples // batch_size)

# บันทึกโมเดลที่ฝึกแล้ว
model.save("face_recognition_model.h5")

print("โมเดลได้รับการฝึกและบันทึกเรียบร้อยแล้ว!")

In [ ]:
#Test 1 Know or Unknow Face
import cv2
import numpy as np
import tensorflow as tf

# โหลดโมเดลที่ฝึกแล้ว
model = tf.keras.models.load_model("face_recognition_model.h5")

# โหลดตัวตรวจจับใบหน้า (Haarcascade)
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# เปิดกล้องวิดีโอ
cap = cv2.VideoCapture(0)

# พารามิเตอร์ของภาพ (ใช้ขนาด 128x128 เพื่อให้ตรงกับโมเดล)
img_height, img_width = 128, 128

while True:
    # อ่านเฟรมจากกล้อง
    ret, frame = cap.read()

    if not ret:
        print("ไม่สามารถอ่านภาพจากกล้องได้")
        break

    # แปลงภาพเป็นสีเทาเพื่อเพิ่มความเร็วในการตรวจจับใบหน้า
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # ตรวจจับใบหน้าในเฟรม
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for x, y, w, h in faces:
        # วาดกรอบสี่เหลี่ยมรอบใบหน้าที่ตรวจพบ
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # ตัดส่วนของใบหน้าออกจากภาพเพื่อใช้โมเดลทำนาย
        face_img = frame[y : y + h, x : x + w]
        face_img = cv2.resize(
            face_img, (img_height, img_width)
        )  # ปรับขนาดภาพให้เหมาะกับโมเดล
        face_img = np.expand_dims(face_img, axis=0)  # เพิ่มมิติที่ 4 ให้กับภาพ
        face_img = face_img / 255.0  # ทำการ normalize

        # ใช้โมเดลทำนายว่าใบหน้าคือใบหน้าที่รู้จักหรือไม่
        prediction = model.predict(face_img)

        # แสดงผลลัพธ์จากการทำนาย
        if prediction[0] > 0.1:
            label = "Known Face"
        else:
            label = "Unknown Face"

        # ใส่ข้อความในกรอบใบหน้า
        cv2.putText(
            frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2
        )

    # แสดงผลเฟรมปัจจุบัน
    cv2.imshow("Face Recognition", frame)

    # กด 'q' เพื่อออกจากลูป
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# ปิดกล้องและปิดหน้าต่าง
cap.release()
cv2.destroyAllWindows()

In [ ]:
#Test 2 Have a signal to respond your Expect Result
import cv2
import numpy as np
import tensorflow as tf

# โหลดโมเดลที่ฝึกแล้ว
model = tf.keras.models.load_model("face_recognition_model.h5")

# โหลดตัวตรวจจับใบหน้า (Haarcascade)
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# เปิดกล้องวิดีโอ
cap = cv2.VideoCapture(0)

# พารามิเตอร์ของภาพ
img_height, img_width = 128, 128

while True:
    # อ่านเฟรมจากกล้อง
    ret, frame = cap.read()

    if not ret:
        print("ไม่สามารถอ่านภาพจากกล้องได้")
        break

    # แปลงภาพเป็นสีเทาเพื่อเพิ่มความเร็วในการตรวจจับใบหน้า
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # ตรวจจับใบหน้าในเฟรม
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    known_face_detected = False

    for x, y, w, h in faces:
        # วาดกรอบสี่เหลี่ยมรอบใบหน้าที่ตรวจพบ
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # ตัดส่วนของใบหน้าออกจากภาพเพื่อใช้โมเดลทำนาย
        face_img = frame[y : y + h, x : x + w]
        face_img = cv2.resize(
            face_img, (img_height, img_width)
        )  # ปรับขนาดภาพให้เหมาะกับโมเดล
        face_img = np.expand_dims(face_img, axis=0)  # เพิ่มมิติที่ 4 ให้กับภาพ
        face_img = face_img / 255.0  # ทำการ normalize

        # ใช้โมเดลทำนายว่าใบหน้าคือใบหน้าที่รู้จักหรือไม่
        prediction = model.predict(face_img)

        # แสดงผลลัพธ์จากการทำนาย
        if prediction[0] > 0.5:  # ปรับ threshold ตามความเหมาะสม
            label = "Known Face"
            known_face_detected = True
        else:
            label = "Unknown Face"

        # ใส่ข้อความในกรอบใบหน้า
        cv2.putText(
            frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2
        )

    # วาดวงกลมที่มุมขวาบนของหน้าต่าง
    if known_face_detected:
        cv2.circle(frame, (frame.shape[1] - 30, 30), 20, (0, 255, 0), -1)  # วงกลมสีเขียว
    else:
        cv2.circle(frame, (frame.shape[1] - 30, 30), 20, (0, 0, 255), -1)  # วงกลมสีแดง

    # แสดงผลเฟรมปัจจุบัน
    cv2.imshow("Face Recognition", frame)

    # กด 'q' เพื่อออกจากลูป
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# ปิดกล้องและปิดหน้าต่าง
cap.release()
cv2.destroyAllWindows()